In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/CS598_PROJECT')

In [4]:
pip install pyhealth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.

In [5]:
# -*- coding: utf-8 -*-
"""create_data_set.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Sh7VQCwkKTK197OEBtSH9gUPvy7ZPdLe
"""

import pandas as pd
from pyhealth.medcode import CrossMap, InnerMap
import pandas as pd

# Function to read the patient CSV file
def read_patient():
    return pd.read_csv("/content/drive/MyDrive/CS598_PROJECT/data/PATIENTS.csv")

# Function to read the visit CSV file
def read_visit():
    return pd.read_csv('/content/drive/MyDrive/CS598_PROJECT/data/ADMISSIONS.csv')

# Function to read the ICD_DIAGNOSES dictionary CSV file
def read_icd9_code():
    return pd.read_csv('/content/drive/MyDrive/CS598_PROJECT/data/D_ICD_DIAGNOSES.csv')

# Function to read the diagnosis CSV file
def read_diagnosis():
    return pd.read_csv("/content/drive/MyDrive/CS598_PROJECT/data/DIAGNOSES_ICD.csv")

# Function to read the medication CSV file
def read_medication():
    return pd.read_csv("/content/drive/MyDrive/CS598_PROJECT/data/PRESCRIPTIONS.csv", low_memory=False)

# Function to preprocess the patient dataframe
def preprocess_patient(df):
    selected_columns = ['SUBJECT_ID','DOB','DOD']
    return df[selected_columns].copy()

# Function to preprocess the visit dataframe
def preprocess_visit(df):
    selected_columns = ['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DEATHTIME']
    return df[selected_columns].copy()

# Function to preprocess the diagnosis dataframe
def preprocess_diagnosis(df):
    selected_columns = ['SUBJECT_ID','HADM_ID','ICD9_CODE']
    return df[selected_columns].copy()

# Function to preprocess the medication dataframe
def preprocess_medication(df):
    selected_columns = ['SUBJECT_ID','HADM_ID','STARTDATE','ENDDATE','DRUG_TYPE','DRUG']
    return df[selected_columns].copy()


In [21]:
## read data set

patient_raw = read_patient()
patient_df = preprocess_patient(patient_raw)

visit_raw = read_visit()
visit_df = preprocess_visit(visit_raw)

diagnosis_raw = read_diagnosis()
diagnosis_df = preprocess_diagnosis(diagnosis_raw)

medication_raw = read_medication()
medication_df = preprocess_medication(medication_raw)

## create ICD9 token2index

icd9cm = InnerMap.load("ICD9CM")

icd9_code_dictionary = read_icd9_code()

ancestors = list(set([j for i in icd9_code_dictionary['ICD9_CODE'] for j in icd9cm.get_ancestors(i) if '.' not in j]))
icd9_token2idx = {ancestors[i] : i for i in range(len(ancestors))}
icd9_idx2token = {i : ancestors[i] for i in range(len(ancestors))}
icd9_token2idx['UNK'] = len(ancestors)
icd9_token2idx['MASK'] = len(ancestors)+1
icd9_token2idx['PAD'] = len(ancestors)+2
icd9_idx2token[len(ancestors)] = 'UNK'
icd9_idx2token[len(ancestors)+1] = 'MASK'
icd9_idx2token[len(ancestors)+2] = 'PAD'

def icd9_ancestor(row):
  row = str(row)
  if len(row) == 0 or row not in icd9cm:
    return row
  df_ancestors = [i for i in icd9cm.get_ancestors(row) if '.' not in i]
  return df_ancestors[0] if len(df_ancestors) > 0 else df_ancestors

def icd9_mapping(row):
  if len(row) == 0 or row not in ancestors:
    return row
  return icd9_token2idx[str(row)]

# Add Age
patient_df['DOB'] = pd.to_datetime(patient_df['DOB']).dt.date
visit_df['ADMITTIME'] = pd.to_datetime(visit_df['ADMITTIME']).dt.date
df = visit_df.merge(patient_df, on='SUBJECT_ID')
df = df.merge(diagnosis_df, on=['SUBJECT_ID', 'HADM_ID'])
df['AGE'] = (df['ADMITTIME'] - df['DOB']).apply(lambda x: int(x.days/365))
subject_id_counts = df.groupby('SUBJECT_ID')['ADMITTIME'].nunique()
# filter out the patient with less than 2 visits
valid_subject_ids = subject_id_counts[subject_id_counts > 1].index
df = df[df['SUBJECT_ID'].isin(valid_subject_ids)].copy()

# Add ICD9_CODE
df['ICD9_CODE_ANCESTOR'] = df['ICD9_CODE'].apply(icd9_ancestor)
df['ICD9_CODE_ANCESTOR_INDEX'] = df['ICD9_CODE_ANCESTOR'].apply(icd9_mapping)

index = [False if isinstance(i, list) else True for i in df['ICD9_CODE_ANCESTOR_INDEX']]
df = df.iloc[index]
df = df.reset_index(drop = True)

# group by date
diagnoses = df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])
diagnoses_grouped = diagnoses.groupby(['SUBJECT_ID', 'ADMITTIME']).agg({'ICD9_CODE_ANCESTOR_INDEX': list, 'AGE': list, 'DOB': 'first'}).reset_index()
diagnoses_grouped.columns = ['SUBJECT_ID', 'ADMITTIME', 'ICD9_CODE', 'AGE', 'DOB']

# diagnoses_grouped.head(20)


In [22]:
token2idx = {'token2idx' :icd9_token2idx, 'idx2token' : icd9_idx2token}

In [23]:
df.head(10)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,DOB,DOD,ICD9_CODE,AGE,ICD9_CODE_ANCESTOR,ICD9_CODE_ANCESTOR_INDEX
0,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,41401,71,414,457
1,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,4111,71,411,664
2,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,4241,71,424,363
3,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,V4582,71,V45,808
4,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,2724,71,272,451
5,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,4019,71,401,1024
6,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,60000,71,600,336
7,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,2082-07-17,NaN,3899,71,389,151
8,23,124321,2157-10-18,2157-10-25 14:00:00,NaN,2082-07-17,NaN,2252,75,225,393
9,23,124321,2157-10-18,2157-10-25 14:00:00,NaN,2082-07-17,NaN,3485,75,348,279


In [24]:
import pickle
with open('/content/drive/MyDrive/CS598_PROJECT/output/token2idx.pkl', 'wb') as pickle_file:
    pickle.dump(token2idx , pickle_file)

In [25]:
diagnoses_final = diagnoses_grouped[["SUBJECT_ID", "ADMITTIME", "DOB", "ICD9_CODE", "AGE"]]

diagnoses_final["NEW_AGE"] = diagnoses_final['AGE'].apply(lambda x: x + [x[0]])
diagnoses_final["NEW_ICD9_CODE"] = diagnoses_final["ICD9_CODE"].apply(lambda x: x + ["SEP"])
# diagnoses_final.head(20)

<ipython-input-25-4e0b3a684b00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnoses_final["NEW_AGE"] = diagnoses_final['AGE'].apply(lambda x: x + [x[0]])
<ipython-input-25-4e0b3a684b00>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnoses_final["NEW_ICD9_CODE"] = diagnoses_final["ICD9_CODE"].apply(lambda x: x + ["SEP"])


In [26]:
diagnoses_grouped_final = diagnoses_final.sort_values(by=['ADMITTIME'], ascending=True).groupby(['SUBJECT_ID']).agg({'NEW_ICD9_CODE': list, 'NEW_AGE': list}).reset_index()
diagnoses_grouped_final["ICD9_CODE"] = diagnoses_grouped_final["NEW_ICD9_CODE"].apply(lambda nested_list: [item for sublist in nested_list for item in sublist])
diagnoses_grouped_final["AGE"] = diagnoses_grouped_final["NEW_AGE"].apply(lambda nested_list: [item for sublist in nested_list for item in sublist])
diagnoses_grouped_final.drop(["NEW_ICD9_CODE", "NEW_AGE"], axis = 1, inplace=True)
diagnoses_grouped_final.columns = ['SUBJECT_ID', 'ICD9_CODE', 'AGE']
diagnoses_grouped_final.head(50)


,SUBJECT_ID,ICD9_CODE,AGE
0,17,"[33, 570, 759, 451, SEP, 857, 353, 766, 570, 9...","[47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 4..."
1,21,"[1010, 766, 314, 582, 837, 858, 859, 52, 528, ...","[87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 8..."
2,23,"[457, 664, 363, 808, 451, 1024, 336, 151, SEP,...","[71, 71, 71, 71, 71, 71, 71, 71, 71, 75, 75, 7..."
3,34,"[1010, 858, 93, 528, 400, 368, 457, 774, SEP, ...","[300, 300, 300, 300, 300, 300, 300, 300, 300, ..."
4,36,"[457, 664, 1024, 1054, 411, 336, 666, 427, SEP...","[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 6..."
5,61,"[477, 1003, 780, 314, 100, 584, 710, 285, 11, ...","[54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55, 5..."
6,67,"[110, 543, 1024, 442, 100, SEP, 442, 1024, 569...","[70, 70, 70, 70, 70, 70, 73, 73, 73, 73]"
7,68,"[93, 858, 858, 582, 877, 584, 892, 543, 100, 4...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4..."
8,84,"[488, SEP, 488, 279, 625, 914, SEP]","[44, 44, 44, 44, 44, 44, 44]"
9,85,"[363, 477, 1023, 110, 55, 691, 411, 336, 774, ...","[71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 7..."


In [20]:
# diagnoses_grouped_final.to_csv('/content/drive/MyDrive/CS598_PROJECT/output/diagnoses.csv', index=False)
# File path to save the pickle file
file_path = '/content/drive/MyDrive/CS598_PROJECT/output/dataset.pkl'

# Save the DataFrame to a pickle file using pandas
diagnoses_grouped_final.to_pickle(file_path)

In [ ]:
# Import necessary libraries
# from pyspark.sql import SparkSession

# Create a SparkSession
#spark = SparkSession.builder \
#   .appName("CSV to DataFrame") \
#    .getOrCreate()

# Read CSV file into a DataFrame
#diagnoses2 = spark.read.csv("/content/drive/MyDrive/CS598_PROJECT/output/diagnoses.csv", header=True, inferSchema=True)

# Register the DataFrame as a temporary view
# diagnoses2.createOrReplaceTempView("csv_table")

# Query the DataFrame using SQL
# diagnoses3 = spark.sql("SELECT * FROM csv_table")

# Show the result
# diagnoses2.show()


In [ ]:
# diagnoses3.show(100)

In [ ]:
# diagnoses4 = diagnoses3.groupby(['SUBJECT_ID', 'ADMITTIME']).agg(F.collect_list('ICD9_CODE_ANCESTOR_INDEX').alias('ICD9_CODE_ANCESTOR_INDEX'), F.collect_list('AGE').alias('AGE'), F.first('DOB').alias('DOB'))

In [ ]:
# diagnoses4.show()

In [ ]:
# from pyspark.sql import Window
# w = Window.partitionBy('SUBJECT_ID').orderBy('ADMITTIME')
# sort and merge ccs and age
# diagnoses9 = diagnoses4.withColumn('ICD9_CODE_ANCESTOR_INDEX', F.collect_list('ICD9_CODE_ANCESTOR_INDEX').over(w)).withColumn('AGE', F.collect_list('AGE').over(w)) #.groupBy('SUBJECT_ID').agg(F.max('ICD9_CODE_ANCESTOR_INDEX').alias('ICD9_CODE_ANCESTOR_INDEX'), F.max('AGE').alias('AGE'))

In [ ]:
# diagnoses9.show()

In [ ]:
# diagnoses4.show()

In [ ]:
# from pyspark.sql.functions import col
# selected_row = diagnoses4.filter(col("SUBJECT_ID") == 34).collect()
# selected_row

In [ ]:
# from pyspark.sql.functions import col
# selected_row = diagnoses9.filter(col("SUBJECT_ID") == 34).collect()
# selected_row